In [ ]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
import os 
os.environ["GROQ_API_KEY"] ="YOUR_API_KEY_HERE"

In [93]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor 
    between a given base currency and a target currency
    """
    url = f'https://v6.exchangerate-api.com/v6/a5fd0285f9c13aaedbad4f6f/pair/{base_currency}/{target_currency}'

    response = requests.get(url)

    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    given a currency conversion rate this function calculates the target currency 
    value from a given base currency value
    """

    return base_currency_value * conversion_rate



In [94]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [95]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1767139202,
 'time_last_update_utc': 'Wed, 31 Dec 2025 00:00:02 +0000',
 'time_next_update_unix': 1767225602,
 'time_next_update_utc': 'Thu, 01 Jan 2026 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 89.8827}

In [96]:
# tool binding
llm = ChatGroq(model = "moonshotai/kimi-k2-instruct-0905")

In [97]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [98]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 usd to inr')]

In [99]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [100]:
ai_message = llm_with_tools.invoke(messages)

In [101]:
messages.append(ai_message)

In [102]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'functions.get_conversion_factor:0',
  'type': 'tool_call'}]

In [103]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [104]:
json.loads(tool_message1.content)['conversion_rate']

89.8827

In [105]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Let me fetch the conversion factor between INR and USD for you.', additional_kwargs={'tool_calls': [{'id': 'functions.get_conversion_factor:0', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 216, 'total_tokens': 256, 'completion_time': 0.113597801, 'completion_tokens_details': None, 'prompt_time': 0.111325965, 'prompt_tokens_details': None, 'queue_time': 0.483058413, 'total_time': 0.224923766}, 'model_name': 'moonshotai/kimi-k2-instruct-0905', 'system_fingerprint': 'fp_05df423bab', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b74c2-7f7d-7581-a392-091bf89c38b7-0', to

In [106]:
llm_with_tools.invoke(messages).content

'Great! The conversion factor from USD to INR is **89.8827**. \n\nNow let me convert 10 USD to INR using this rate:'

In [107]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

ImportError: cannot import name 'initialize_agent' from 'langchain.agents' (c:\CODE PLAYGROUND\Gen AI using LangChain\Tools in Langchain\.venv\Lib\site-packages\langchain\agents\__init__.py)

In [ ]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})